Батчинг (batch processing) - это техника обработки данных группами (пакетами), а не по одному.


При кросс-валидации данные делятся на несколько частей, часть из которых используется обучения, а оставшаяся часть — для проверки. Это позволяет получить надежную оценку, так как для проверки используются данные, которые модель еще не видела.

## Типы кросс-валидации

### Валидация на отложенных данных (Hold-Out Cross-Validation)
Самый простой способ: данные делят на тренировочные и тестовые. Первые используются для обучения модели, а тестовые — для оценки производительности. 

In [ ]:
#Hold-Out Cross-Validation
import numpy as np
from sklearn.model_selection import train_test_split
 
X, y = np.arange(1000).reshape((500, 2)), np.arange(500)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

Если у вас достаточно данных, лучше всегда предусматривать также валидационное множество. Это третье множество данных, которое позволяет безопасно оценить качество модели и предотвратить её переобучение.

Разбиение данных на тренировочное, тестовое и валидационное множество в библиотеке sklearn:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
 
X, y = np.arange(1000).reshape((500, 2)), np.arange(500)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.1, 
    random_state=42
)

Если вы перебираете какие-то модели для вашей задачи, то оптимизировать их качество стоит на валидационном множестве, а окончательно сравнивать модели на тестовом множестве.

Оптимизация модели может включать:

- подбор гиперпараметров;
- подбор архитектуры (если речь идёт о нейросетях);
- подбор оптимального трешхолда для максимизации значений целевой метрики — например, вы делаете двуклассовую классификацию, а модель выдаёт - непрерывные значения от 0 до 1, которые нужно бинаризовать так, чтобы получить максимальный скор по F1.

### k-Fold (K-Fold Cross-Validation)
Метод k-Fold чаще всего имеют в виду, когда говорят о кросс-валидации. Он является обобщением метода hold-out и представляет из себя следующий алгоритм:

- Фиксируется некоторое целое число k (обычно от 5 до 10), меньше числа семплов в датасете.

- Датасет разбивается на k одинаковых частей (в последней части может быть меньше семплов, чем в остальных). Эти части называются фолдами.

- Далее происходит k итераций, во время каждой из которых один фолд выступает в роли тестового множества, а объединение остальных — в роли тренировочного. Модель учится на k−1 фолде и тестируется на оставшемся.

- Финальный скор модели либо получается усреднением k получившихся тестовых результатов, либо измеряется на отложенном тестовом множестве, не участвовавшем в кросс-валидации.


Данные делятся на несколько равных частей (К): одна используется для проверки, остальные — для обучения. Процесс повторяется столько раз, на сколько частей мы поделили данные. Каждый блок должен быть использован как тестовый набор.

После окончания обучения результаты тестов приводят к среднему значению. Это позволяет получить более надежную оценку производительности.

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
 
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=2)
 
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
'''
result:
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1] TEST: [2 3]
'''

В sklearn есть также метод cross_val_score, который принимает на вход классификатор, данные и способ разбиения данных (либо число фолдов) и возвращает результаты кросс-валидации:


In [ ]:
from sklearn.model_selection import cross_val_score
 
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)
'''
result:
array([0.96..., 1. , 0.96..., 0.96..., 1. ])
'''

### стратификация
стратификация — разбиение на трейн и тест, сохраняющее исходное соотношение классов
Стратифицированная K-блочная кросс-валидация (Stratified K-Fold Cross-Validation)
Проводится аналогично K-блочной кросс-валидации, но с учетом пропорции классов в каждом блоке. Используется, когда данные имеют несбалансированные классы.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
 
X, y = np.arange(1000).reshape((500, 2)), np.random.choice(4, size=500, p=[0.1, 0.2, 0.3, 0.4])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)


### Leave-one-out
Метод leave-one-out (LOO) — частный случай метода k-Fold: в нём каждый фолд состоит ровно из одного семпла.

In [ ]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
 
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([1, 2, 3])
loo = LeaveOneOut()
 
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
'''
result:
TRAIN: [1 2] TEST: [0]
TRAIN: [0 2] TEST: [1]
TRAIN: [0 1] TEST: [2]
'''


Метод stratified k-Fold — это метод k-Fold, использующий стратификацию при разбиении на фолды: каждый фолд содержит примерно такое же соотношение классов, как и всё исходное множество. Такой подход может потребоваться в случае, например, очень несбалансированного соотношения классов, когда при обычном случайном разбиении некоторые фолды могут либо вообще не содержать семплов каких-то классов, либо содержать их слишком мало.

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
 
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(n_splits=2)
 
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
'''
result:
TRAIN: [1 3] TEST: [0 2]
TRAIN: [0 2] TEST: [1 3]
'''


Основные методы кросс-валидации:

- Hold-out: простое разделение на тренировочное, тестовое и валидационное подмножества. Критически важно перемешивать данные, чтобы предотвратить зависимость качества обучения от порядка данных. Валидационное множество используется для подбора гиперпараметров, а финальная оценка — на тесте.
- k-Fold: датасет делится на k частей, модель обучается и тестируется k раз. Даёт более надёжную оценку, чем hold-out, но требует больше ресурсов. Подходит для небольших данных.
- Stratified k-Fold: сохраняет распределение классов в фолдах, важно для несбалансированных данных (например, спам-фильтрация).
- Leave-one-out (LOO): частный случай k-Fold, где каждый фолд — один образец. Применяется при очень малом количестве данных.

### MapReduce
в основе работы MapReduce лежат Map и Reduce. 

- Map: Этот шаг преобразует входные данные в набор пар «ключ-значение». Каждая функция Map работает независимо и параллельно с другими, обрабатывая свою часть входной информации. В результате каждого вызова функции Map генерируется набор промежуточных пар «ключ-значение».

- Reduce: На этом шаге происходит работа с данными, сгруппированными по ключу, и преобразование их в набор выходных значений. Reduce работает после того, как все функции Map завершили свою работу. Она принимает ключ и множество значений, сопоставленных этому ключу, и выдает набор выходных значений (обычно одно значение для каждого уникального ключа).

In [2]:
from collections import defaultdict
from typing import List, Tuple, Any, Callable
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

class SimpleMapReduce:
    def __init__(self, num_workers=None):
        self.num_workers = num_workers or multiprocessing.cpu_count()
    
    def map_reduce(self, 
                   data: List,
                   mapper: Callable,
                   reducer: Callable,
                   shuffle_sort: bool = True) -> List:
        """
        Основная функция MapReduce
        """
        # 1. Map этап
        mapped = self._map(data, mapper)
        
        # 2. Shuffle этап (группировка по ключам)
        shuffled = self._shuffle(mapped, shuffle_sort)
        
        # 3. Reduce этап
        reduced = self._reduce(shuffled, reducer)
        
        return reduced
    
    def _map(self, data: List, mapper: Callable) -> List[Tuple]:
        """Параллельное выполнение Map"""
        with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            results = list(executor.map(mapper, data))
        # Преобразуем в плоский список
        flat_results = []
        for result in results:
            flat_results.extend(result)
        return flat_results
    
    def _shuffle(self, mapped_data: List[Tuple], sort: bool) -> dict:
        """Группировка данных по ключам"""
        shuffled = defaultdict(list)
        for key, value in mapped_data:
            shuffled[key].append(value)
        
        if sort:
            # Сортировка ключей (опционально)
            shuffled = dict(sorted(shuffled.items()))
        
        return shuffled
    
    def _reduce(self, shuffled_data: dict, reducer: Callable) -> List:
        """Параллельное выполнение Reduce"""
        results = []
        with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            # Подготавливаем задачи
            tasks = [(key, values) for key, values in shuffled_data.items()]
            # Выполняем reduce
            reduced = executor.map(
                lambda task: (task[0], reducer(task[0], task[1])), 
                tasks
            )
            results = list(reduced)
        
        return results

In [3]:
# Пример данных
documents = [
    "машинное обучение ",
    "глубокое обучение часть машинного обучения",
    "машинное обучение и deep learning"
]

def word_count_mapper(doc: str) -> List[Tuple]:
    """Map: разбиваем документ на слова"""
    words = doc.lower().split()
    return [(word, 1) for word in words]

def word_count_reducer(key: str, values: List[int]) -> int:
    """Reduce: суммируем количества"""
    return sum(values)

# Запуск MapReduce
mr = SimpleMapReduce()
result = mr.map_reduce(
    data=documents,
    mapper=word_count_mapper,
    reducer=word_count_reducer
)

print("Word Count Results:")
for word, count in sorted(result, key=lambda x: x[1], reverse=True):
    print(f"  {word}: {count}")

Word Count Results:
  обучение: 3
  машинное: 2
  deep: 1
  learning: 1
  глубокое: 1
  и: 1
  машинного: 1
  обучения: 1
  часть: 1


In [ ]:

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
X = data.drop('target', axis=1)
y = data['target']
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]
model = LogisticRegression)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
# Повторяем то же, что и в k-fold


from sklearn.model_selection import TimeSeriesSplit
# Сортируем по дате
data = data.sort_values(by='date')
X = data.drop(['target', 'date'], axis=1)
y = data['target']
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X):
# Повторяем то же, что и в k-fold


from sklearn.model _selection import LeaveOneOut
loo = LeaveOneOut()
for train_index, test_index in loo.split(X):
# Этот метод может быть очень время затратным на больших
# Повторяем то же, что и в k-fold


from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
# Повторяем то же, что и в k-fold